# Tutorial - Wind Power Prediction Using Gaussian Processes

***Machine Learning for Renewable Energy Systems***

**Timo Lübbing**

**University of Tübingen**

## Introduction

This tutorial explores the application of Gaussian Processes (GP) in forecasting wind power generation. Through a series of code examples and explanations, you'll gain insights into data preprocessing, feature engineering, and the training and evaluation of GP models for wind energy prediction.

We'll also delve into forecasting the combined output of off-shore and on-shore wind power. Additionally, we'll examine how different weather input features influence the model's performance, providing a deeper understanding of their impact on predictive accuracy.